In [16]:

import json
import tweepy
import pandas as pd
from tweepy import OAuthHandler
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import time
import re

#Analysis imports
import glob
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from collections import Counter
import numpy as np

analyzer = SentimentIntensityAnalyzer()

In [9]:
analyzer = SentimentIntensityAnalyzer()

In [3]:
analyzer = SentimentIntensityAnalyzer()
res = analyzer.polarity_scores('Vader is a bad guy by I love this analyzer.')
res

{'neg': 0.255, 'neu': 0.438, 'pos': 0.307, 'compound': 0.1779}

In [4]:
con_key = 'uz4wSx2vt2PlKbWCOlIpKbBZo'
con_secret = 'lq61FxYRvJPwLjjRlUf0rSTsdXf86iijHYeGfrUdi7EhZzWsWv'
acc_token = '1038510225786122247-zLnTCjChQ0rLUHevueL0CN4ygdY1Me'
acc_token_secret = 'W7lqUevzXRpzI4zQA6niMDR85r9SCo3sCQGaPA4wg6DS6'

In [5]:
analyzer = SentimentIntensityAnalyzer()
res = analyzer.polarity_scores('Vader is a bad guy by I love this analyzer.')

In [6]:
auth = OAuthHandler(con_key, con_secret)
auth.set_access_token(acc_token, acc_token_secret)
new = tweepy.API(auth)

In [7]:
results = new.search(q="@FLGovScott" , count = 2 , lang='en' ,  geocode = '27.967042,-81.471955,320km' , tweet_mode = 'extended')



## This is the Begining

In [8]:
bagOwords = ['election' , 'senate' , 'race' , 'november' , '6' , 'gop' , 'dnc' , 'win' , 'lose']
analyzer = SentimentIntensityAnalyzer()
def Senter(tweets , Cadidate , DF):
    for tweet in tweets:
        sentiment = '0'
        retweet = '0'
        relivent = 0
        
        if tweet.full_text[:2] == 'RT':
            retweet = '1'
        
        lower_txt = tweet.full_text.lower()
        for word in bagOwords:
            if word in lower_txt:
                relivent = 1
        
        if tweet.lang == 'en':
            #send to english sentiment analyser
            sentiment = analyzer.polarity_scores(tweet.full_text)
        else:
            #send to this sentiment analyser
            sentiment = analyzer.polarity_scores(tweet.full_text)
        DF = DF.append({'Candidate': '%s' % (Cadidate) , 'Time' : tweet.created_at ,
               'Text': tweet.full_text , 'UserLocation': tweet.user.location , 
               'Coordinates': tweet.coordinates , 'Retweeted_count': tweet.retweeted_count , 
               'IsReTweet': retweet ,'Language' : tweet.lang , 'Sentiment': sentiment ,
               'Relivent' : relivent } , ignore_index = True)

    return DF

In [9]:
#Keep in mind we are only pulling english tweets
def Searcher(Handle , Cadidate , GeoCode , DF):
    tweets = new.search(q='%s' % (Handle), lang = 'en' , count = consume_count  , geocode = '%s' % (GeoCode) , tweet_mode = 'extended')     
    if len(tweets) == 0:
        time.sleep(60 * 15)
        tweets = new.search(q='%s' % (Handle), lang = 'en' , count = consume_count ,  geocode = '%s' % (GeoCode) , tweet_mode = 'extended')    
    DF = Senter(tweets , Cadidate , DF)
    return DF

In [10]:
columns = ['Candidate' , 'Time' , 'Text' , 'UserLocation' , 'Coordinates' , 'Retweeted_count' , 'IsReTweet' , 'Language' , 'Sentiment' , 'Relivent' ]
AZ_path_name = 'iCloud Drive/Desktop/Politter/AZOutput'
FL_path_name = 'iCloud Drive/Desktop/Politter/FLOutput'+'%s'+ '.csv' % (i)
FL_Radius = '27.967042,-81.471955,320km'
AZ_Radius = '34.383026,-111.694738,400km'
consume_count = 15
analyzer = SentimentIntensityAnalyzer()
FL = pd.DataFrame(columns = columns)
AZ = pd.DataFrame(columns = columns)

while True:
    
    if time.localtime().tm_hour == 24:
        FL.to_csv(FL_path_name , sep = ',')
        AZ.to_csv(AZ_path_name , sep = ',')
        i+=1
        
    FL = Searcher("@FLGovScott" , 'GOP:RICK SCOTT' , FL_Radius , FL)
    FL = Searcher("@SenBillNelson" , 'DEM:BILL NELSON' , FL_Radius , FL)
    
    AZ = Searcher("@RepMcSally" , 'GOP:MARTHA MCSALLY' , AZ_Radius , AZ)
    AZ = Searcher("@RepSinema" , 'DEM:KYRSTEN SINEMA' , AZ_Radius , AZ)
    

SyntaxError: invalid syntax (<ipython-input-10-6eacad91577d>, line 3)

## END

---

---

---

---

# PROCESSING 

In [4]:
path = '/Users/pbfile/Desktop/TextMining' # use your path
allFiles = glob.glob(path + "/*.csv")
pol_df = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_, header=0)
    list_.append(df)
pol_df = pd.concat(list_)
pol_df.drop(['Unnamed: 0'] , inplace = True , axis = 1)
pol_df.reset_index(inplace=True , drop= True)

In [5]:
pol_df

,Candidate,Time,Text,UserLocation,Coordinates,Retweeted_count,IsReTweet,Language,Sentiment,Relivent
0,GOP:MARTHA MCSALLY,2018-10-02 19:42:39,RT @MicheleWilcox53: To all my AZ people:\r\nI...,USA,NaN,1,1,en,"{'neg': 0.0, 'neu': 0.737, 'pos': 0.263, 'comp...",0
1,GOP:MARTHA MCSALLY,2018-10-02 19:38:17,@thehill @RepAndyBiggsAZ @JeffFlake @dougducey...,"Gilbert, AZ",NaN,0,0,en,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",1
2,GOP:MARTHA MCSALLY,2018-10-02 19:37:46,@RepMcSally proud to support in any way I can....,"Phoenix, AZ",NaN,0,0,en,"{'neg': 0.0, 'neu': 0.651, 'pos': 0.349, 'comp...",1
3,GOP:MARTHA MCSALLY,2018-10-02 19:37:35,@thehill Trumpism is ruining us. Trumpism is ...,"Gilbert, AZ",NaN,0,0,en,"{'neg': 0.204, 'neu': 0.796, 'pos': 0.0, 'comp...",1
4,GOP:MARTHA MCSALLY,2018-10-02 19:29:20,@RepMcSally \r\nMartha McSally for Arizona.!! ...,"Arizona, USA",NaN,0,0,en,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0
5,GOP:MARTHA MCSALLY,2018-10-02 19:25:34,@KarleenRG @RepMcSally @RepSinema Translation:...,"Phoenix, AZ",NaN,0,0,en,"{'neg': 0.095, 'neu': 0.905, 'pos': 0.0, 'comp...",0
6,GOP:MARTHA MCSALLY,2018-10-02 19:02:12,"@GailRule1 @RepMcSally @Liberty4Rox ""I didn’t ...","Phoenix, AZ",NaN,0,0,en,"{'neg': 0.124, 'neu': 0.876, 'pos': 0.0, 'comp...",0
7,GOP:MARTHA MCSALLY,2018-10-02 18:59:52,"@KarleenRG @RepMcSally @RepSinema ""Oppressing ...","Phoenix, AZ",NaN,0,0,en,"{'neg': 0.192, 'neu': 0.808, 'pos': 0.0, 'comp...",1
8,GOP:MARTHA MCSALLY,2018-10-02 18:58:02,@KarleenRG @RepMcSally @RepSinema There is no ...,"Phoenix, AZ",NaN,0,0,en,"{'neg': 0.319, 'neu': 0.681, 'pos': 0.0, 'comp...",0
9,GOP:MARTHA MCSALLY,2018-10-02 18:23:40,RT @RepMcSally: Small businesses employ our fa...,Texas,NaN,21,1,en,"{'neg': 0.0, 'neu': 0.596, 'pos': 0.404, 'comp...",0


In [35]:
pol_df.dtypes

Candidate           object
Time                object
Text                object
UserLocation        object
Coordinates        float64
Retweeted_count      int64
IsReTweet            int64
Language            object
Sentiment           object
Relivent             int64
dtype: object

In [41]:
pol_df['RT_Remove'] = '0'
pol_df['Hash_Remove'] = '0'
pol_df['At_Remove'] = '0'
pol_df['Hash_At_Remove'] = '0'
for row in pol_df.itertuples():
    if row.IsReTweet == 1:
        tweet = row.Text[2:]
    else:
        tweet = row.Text
    
    pol_df.at[row.Index , 'RT_Remove'] = tweet
    pol_df.at[row.Index , 'Hash_Remove'] = (" ".join(filter(lambda x:x[0]!='#', tweet.split())))
    pol_df.at[row.Index , 'At_Remove'] = (" ".join(filter(lambda x:x[0]!='@', tweet.split())))
    
    remove_hyp = (" ".join(filter(lambda x: x[0]!='#' and x[0]!='@' and x[0:4]!='&amp', tweet.split())))
    pol_df.at[row.Index , 'Hash_At_RT_Remove'] = (re.sub(r"\bhttp\S+\b", "", remove_hyp))
    
    

In [49]:
for row in pol_df.itertuples():
    print(row.Sentiment)
    print(row.Hash_Remove)
    print('\n')
    print(row.Hash_At_RT_Remove)
    print(analyzer.polarity_scores(row.Hash_At_RT_Remove))
    print(row.Candidate)
    print('\n')
    print('\n')

{'neg': 0.0, 'neu': 0.737, 'pos': 0.263, 'compound': 0.7579}
@MicheleWilcox53: To all my AZ people: If Grant Woods, a major McCain supporter &amp; backer from way back (1985) is supporting Sinema and d…


To all my AZ people: If Grant Woods, a major McCain supporter backer from way back (1985) is supporting Sinema and d…
{'neg': 0.0, 'neu': 0.706, 'pos': 0.294, 'compound': 0.7579}
GOP:MARTHA MCSALLY




{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
@thehill @RepAndyBiggsAZ @JeffFlake @dougducey @DebbieLesko @votewarren @JDMesnard @RepGosar @RepMcSally @GOPChairwoman @SpeakerRyan @senatemajldr @westerncaucus @freedomcaucus @RepSinema @RepRaulGrijalva The will wash away the $hit of on Election Day @RepRubenGallego


The will wash away the $hit of on Election Day
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
GOP:MARTHA MCSALLY




{'neg': 0.0, 'neu': 0.651, 'pos': 0.349, 'compound': 0.8805}
@RepMcSally proud to support in any way I can. Do great things in the Senate for st

{'neg': 0.188, 'neu': 0.812, 'pos': 0.0, 'compound': -0.4588}
DEM:BILL NELSON




{'neg': 0.136, 'neu': 0.864, 'pos': 0.0, 'compound': -0.4588}
@craigtimes: Not at all related to @PolitiFact finding that 9 of his 10 ads attacking @SenBillNelson were false. https://t.co/Y…


Not at all related to finding that 9 of his 10 ads attacking were false. …
{'neg': 0.188, 'neu': 0.812, 'pos': 0.0, 'compound': -0.4588}
DEM:BILL NELSON




{'neg': 0.143, 'neu': 0.857, 'pos': 0.0, 'compound': -0.4588}
Not at all related to @PolitiFact finding that 9 of his 10 ads attacking @SenBillNelson were false. https://t.co/YKxJl9e1Pj https://t.co/QxaoTejqXe


Not at all related to finding that 9 of his 10 ads attacking were false.  
{'neg': 0.188, 'neu': 0.812, 'pos': 0.0, 'compound': -0.4588}
DEM:BILL NELSON




{'neg': 0.145, 'neu': 0.761, 'pos': 0.093, 'compound': -0.126}
It’s time for @SenBillNelson to leave the Washington swamp. He’s part of the problem. Let’s show our support and elect @FLGovScott as ou

{'neg': 0.0, 'neu': 0.779, 'pos': 0.221, 'compound': 0.6249}
DEM:KYRSTEN SINEMA




{'neg': 0.0, 'neu': 0.812, 'pos': 0.188, 'compound': 0.6249}
@brahmresnik: THE MONEY POLL Proof 1 of top 3 Senate races in US: TV ad spending tops $40 million. Obliterates any previous e…


THE MONEY POLL Proof 1 of top 3 Senate races in US: TV ad spending tops $40 million. Obliterates any previous e…
{'neg': 0.0, 'neu': 0.779, 'pos': 0.221, 'compound': 0.6249}
DEM:KYRSTEN SINEMA




{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
@Gambo987: You never know who will pop into the KTAR studios ⁦@RepMcSally⁩ https://t.co/QGemMyDBpW


You never know who will pop into the KTAR studios ⁦@RepMcSally⁩ 
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
GOP:MARTHA MCSALLY




{'neg': 0.087, 'neu': 0.913, 'pos': 0.0, 'compound': -0.2023}
@CharlesVanCott: @Doodisgirl @tweetybird2009 Opponents of @RepMcSally have run an ad here in Arizona complaining she votes too often wit…


Opponents of have run an ad he


{'neg': 0.058, 'neu': 0.849, 'pos': 0.093, 'compound': 0.296}
@sdbfo @FLGovScott @FLSERT Good to see fellow Floridians showing concern for one another. Oh wait your not showing concern for the ppl in the panhandle your just talking shit to our GOV. That's real classy of u, did u get all that class in France or Brazil? U may need to go back to France for more lessons


Good to see fellow Floridians showing concern for one another. Oh wait your not showing concern for the ppl in the panhandle your just talking shit to our GOV. That's real classy of u, did u get all that class in France or Brazil? U may need to go back to France for more lessons
{'neg': 0.061, 'neu': 0.842, 'pos': 0.098, 'compound': 0.296}
GOP:RICK SCOTT




{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
@sdbfo @FLGovScott @FLSERT Grow up


Grow up
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
GOP:RICK SCOTT




{'neg': 0.0, 'neu': 0.928, 'pos': 0.072, 'compound': 0.1779}
@FLGovScott: I’ve just been briefe

I engaged w/@realdonaldtrump in yesterday’s meeting to push for real solutions to seal secure the border. I a…
{'neg': 0.0, 'neu': 0.657, 'pos': 0.343, 'compound': 0.7003}
GOP:MARTHA MCSALLY




{'neg': 0.338, 'neu': 0.662, 'pos': 0.0, 'compound': -0.8481}
@RepMcSally: With 134,000 jobs added in September, the unemployment rate is the lowest it's been in 49 years. I have no doubt that @GOP…


With 134,000 jobs added in September, the unemployment rate is the lowest it's been in 49 years. I have no doubt that
{'neg': 0.375, 'neu': 0.625, 'pos': 0.0, 'compound': -0.8481}
GOP:MARTHA MCSALLY




{'neg': 0.338, 'neu': 0.662, 'pos': 0.0, 'compound': -0.8481}
@RepMcSally: With 134,000 jobs added in September, the unemployment rate is the lowest it's been in 49 years. I have no doubt that @GOP…


With 134,000 jobs added in September, the unemployment rate is the lowest it's been in 49 years. I have no doubt that
{'neg': 0.375, 'neu': 0.625, 'pos': 0.0, 'compound': -0.8481}
GOP:MARTHA MCSALLY






{'neg': 0.0, 'neu': 0.868, 'pos': 0.132, 'compound': 0.4404}
@DabbleEndeavors: Thanks @SenBillNelson for showing Florida voters that you will oppose our president at all costs, regardless of merit…


Thanks for showing Florida voters that you will oppose our president at all costs, regardless of merit…
{'neg': 0.0, 'neu': 0.847, 'pos': 0.153, 'compound': 0.4404}
DEM:BILL NELSON




{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
@FLGovScott: We are currently monitoring a weather system over the western Caribbean. As we continue to watch this storm, it’s important…


We are currently monitoring a weather system over the western Caribbean. As we continue to watch this storm, it’s important…
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
GOP:RICK SCOTT




{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
@FLGovScott: We are currently monitoring a weather system over the western Caribbean. As we continue to watch this storm, it’s important…


We are currently monitorin



.@SenBillNelson's camp on claim Nelson's "trying to back out" of the debate: "It’s 1,000% false,…
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
GOP:RICK SCOTT




{'neg': 0.0, 'neu': 0.674, 'pos': 0.326, 'compound': 0.8271}
@FLGovScott: .@MyFWC scientists do a great job every day. We’ve provided nearly $2.2 million to test innovative solutions to mitigate re…


.@MyFWC scientists do a great job every day. We’ve provided nearly $2.2 million to test innovative solutions to mitigate re…
{'neg': 0.0, 'neu': 0.648, 'pos': 0.352, 'compound': 0.8271}
GOP:RICK SCOTT




{'neg': 0.244, 'neu': 0.51, 'pos': 0.246, 'compound': 0.0065}
Thanks @FLGovScott your policies caused this mess https://t.co/hY5QyLs7tt


Thanks your policies caused this mess 
{'neg': 0.294, 'neu': 0.409, 'pos': 0.297, 'compound': 0.0065}
GOP:RICK SCOTT




{'neg': 0.0, 'neu': 0.8, 'pos': 0.2, 'compound': 0.5423}
@TroyKinsey: This is a false narrative created by @FLGovScott, according to the @SenBillNelson campaign. 

# BREAK DOWN OF ReTweets

In [51]:
for row in pol_df.itertuples():
    if row.IsReTweet == 0:
        print(row.Sentiment)
        print(row.Hash_Remove)
        print('\n')
        print(row.Hash_At_RT_Remove)
        print(analyzer.polarity_scores(row.Hash_At_RT_Remove))
        print(row.Candidate)
        print('\n')
        print('\n')

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
@thehill @RepAndyBiggsAZ @JeffFlake @dougducey @DebbieLesko @votewarren @JDMesnard @RepGosar @RepMcSally @GOPChairwoman @SpeakerRyan @senatemajldr @westerncaucus @freedomcaucus @RepSinema @RepRaulGrijalva The will wash away the $hit of on Election Day @RepRubenGallego


The will wash away the $hit of on Election Day
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
GOP:MARTHA MCSALLY




{'neg': 0.0, 'neu': 0.651, 'pos': 0.349, 'compound': 0.8805}
@RepMcSally proud to support in any way I can. Do great things in the Senate for state of Arizona and the nation! https://t.co/JEKTeYZ1mH


proud to support in any way I can. Do great things in the Senate for state of Arizona and the nation! 
{'neg': 0.0, 'neu': 0.625, 'pos': 0.375, 'compound': 0.8805}
GOP:MARTHA MCSALLY




{'neg': 0.204, 'neu': 0.796, 'pos': 0.0, 'compound': -0.7506}
@thehill Trumpism is ruining us. Trumpism is a cancer in America. A disease. A Sickenss. @RepAndyBigg

{'neg': 0.0, 'neu': 0.488, 'pos': 0.512, 'compound': 0.636}
@PrincipalHayden @FLGovScott @pbcsd @Wellingtonflgov Congratulations!


Congratulations!
{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.636}
GOP:RICK SCOTT




{'neg': 0.0, 'neu': 0.589, 'pos': 0.411, 'compound': 0.943}
Congratulations to our AMAZINGLY TALENTED students who earned PERFECT SCORES on FSA’s and EOC’s! surrounded by greatness! @FLGovScott @pbcsd @Wellingtonflgov https://t.co/sWelySapLc


Congratulations to our AMAZINGLY TALENTED students who earned PERFECT SCORES on FSA’s and EOC’s! surrounded by greatness! 
{'neg': 0.0, 'neu': 0.5, 'pos': 0.5, 'compound': 0.943}
GOP:RICK SCOTT




{'neg': 0.131, 'neu': 0.79, 'pos': 0.079, 'compound': -0.1779}
@Stewartmoore How is this happening. A SCOTUS nominee with zero temperament, clear partisan bias, and blatant perjury moments in front of the senate committee. @marcorubio @SenBillNelson do what’s right.


How is this happening. A SCOTUS nominee with zero temperament, cl

{'neg': 0.0, 'neu': 0.77, 'pos': 0.23, 'compound': 0.4574}
DEM:KYRSTEN SINEMA




{'neg': 0.139, 'neu': 0.861, 'pos': 0.0, 'compound': -0.7159}
@dougducey AZ has 1 of the highest poverty rates in the US. And we have a very very low ranked K-12 education system. What are you doing to fix this ? @RepAndyBiggsAZ @JeffFlake @DebbieLesko @votewarren @JDMesnard @RepGosar @RepMcSally @RepRaulGrijalva @RepSinema @dg4az @RepRubenGallego


AZ has 1 of the highest poverty rates in the US. And we have a very very low ranked K-12 education system. What are you doing to fix this ?
{'neg': 0.193, 'neu': 0.807, 'pos': 0.0, 'compound': -0.7159}
DEM:KYRSTEN SINEMA




{'neg': 0.103, 'neu': 0.8, 'pos': 0.097, 'compound': -0.0516}
@imaginista111 @Maestro_ZMan @BeverlyFolkers @EdKrassen @SenBillNelson @RepJackyRosen @SenatorBaldwin @clairecmc @SenatorHeitkamp @SenDonnelly @RepSinema @PhilBredesen @BetoORourke @tedcruz Such a good guy, burglarizing here and fleeing from a DWI accident scene there


Such a g

## Tokenizing the removed @, #, RT, HyperLink list

In [43]:

pol_df['Removed_LOWER'] = pol_df.apply(lambda x: x['Hash_At_RT_Remove'].lower(), axis=1).apply(np.array)


In [44]:

pol_df['Tokenized'] = pol_df.apply(lambda x: word_tokenize(x['Removed_LOWER']), axis=1).apply(np.array)


In [45]:
word_lst = []
word_counter = {}

for row in pol_df.itertuples():
    for word in row.Tokenized:
        word_lst.append(word)

        
for word in word_lst:
    if word in word_counter:
        word_counter[word] += 2
    else:
        word_counter[word] = 1        

In [46]:
word_counter

{'to': 1191,
 'all': 133,
 'my': 125,
 'az': 73,
 'people': 101,
 ':': 283,
 'if': 153,
 'grant': 3,
 'woods': 11,
 ',': 921,
 'a': 857,
 'major': 3,
 'mccain': 59,
 'supporter': 3,
 'backer': 3,
 'from': 89,
 'way': 45,
 'back': 55,
 '(': 65,
 '1985': 3,
 ')': 71,
 'is': 751,
 'supporting': 31,
 'sinema': 15,
 'and': 807,
 'd…': 3,
 'the': 1761,
 'will': 219,
 'wash': 13,
 'away': 15,
 '$': 141,
 'hit': 15,
 'of': 739,
 'on': 315,
 'election': 53,
 'day': 51,
 'proud': 17,
 'support': 75,
 'in': 1061,
 'any': 107,
 'i': 683,
 'can': 117,
 '.': 2561,
 'do': 179,
 'great': 59,
 'things': 23,
 'senate': 195,
 'for': 797,
 'state': 61,
 'arizona': 39,
 'nation': 3,
 '!': 601,
 'trumpism': 15,
 'ruining': 7,
 'us': 153,
 'cancer': 7,
 'america': 33,
 'disease': 17,
 'sickenss': 7,
 'martha': 47,
 'mcsally': 69,
 'arizona.': 3,
 'translation': 7,
 'have': 339,
 'no': 239,
 'clue': 7,
 'what': 127,
 'god': 51,
 "'s": 397,
 'word': 29,
 'says': 69,
 'jesus': 19,
 'always': 55,
 'used': 25,
 '

In [38]:
popular_words = sorted(word_counter , key = word_counter.get , reverse = True)

# NOTE: Find it intereisting the 134,000 number is in here --> ReTweets are messing with this

In [81]:
stop_bag = popular_words[:150]

In [82]:
stop_bag

['.',
 'the',
 'to',
 'in',
 ',',
 'a',
 'and',
 'for',
 ':',
 'is',
 'of',
 'you',
 'i',
 '’',
 '!',
 'https',
 'that',
 "'s",
 'we',
 'it',
 'vote',
 'this',
 'have',
 'on',
 'are',
 's',
 'with',
 'not',
 'as',
 'be',
 'your',
 'no',
 'she',
 '?',
 'will',
 'by',
 'senate',
 'florida',
 'he',
 'do',
 'his',
 'been',
 'but',
 't',
 'if',
 'us',
 'years',
 '$',
 'was',
 'who',
 'million',
 '@',
 'all',
 '...',
 "n't",
 'what',
 'kavanaugh',
 'my',
 'over',
 'they',
 'at',
 'jobs',
 'can',
 'has',
 'more',
 'her',
 'now',
 'like',
 'fl',
 '3',
 'need',
 'any',
 'an',
 'doubt',
 '``',
 'people',
 'just',
 'or',
 '134,000',
 'added',
 'september',
 'unemployment',
 'rate',
 'lowest',
 '49',
 'our',
 'up',
 'how',
 'here',
 'scott',
 'thank',
 'another',
 'voting',
 'about',
 'from',
 'senator',
 'poll',
 '1',
 'nelson',
 'money',
 'never',
 "''",
 'too',
 'could',
 'false',
 '2nd',
 'storm',
 'ad',
 'history',
 'should',
 'so',
 'red',
 'top',
 'support',
 'see',
 'am',
 'let',
 'az',
 '

# HOW MANY STOP WORDS DO YOU THINK IS APPROPORATE

In [83]:

stop_bag.remove('not')
stop_bag.remove('no')
stop_bag.remove('false')
stop_bag.remove('support')
stop_bag.remove('good')
stop_bag.remove('against')
stop_bag.remove('for')



In [84]:
def My_stop(words):
    filter_words = []
    for word in words:
        if word not in stop_bag:
              filter_words.append(word)
    return filter_words

In [85]:

pol_df['MY_Stopped'] = pol_df.apply(lambda x: My_stop(x['Tokenized']), axis=1).apply(np.array)


In [88]:
for row in pol_df.itertuples():
    if row.IsReTweet == 0:
        print(row.Text)
        print(row.Sentiment)
        print('\n')
        print(row.Hash_At_RT_Remove)
        print(analyzer.polarity_scores(joined_res))
        print('\n')
        joined_res =  ' '.join(row.MY_Stopped)
        print(joined_res)
        print(analyzer.polarity_scores(joined_res))
        print(row.Candidate)
        print('\n')
        print('\n')
        print('\n')

@thehill @RepAndyBiggsAZ @JeffFlake @dougducey @DebbieLesko @votewarren @JDMesnard @RepGosar @RepMcSally @GOPChairwoman @SpeakerRyan @senatemajldr @westerncaucus @freedomcaucus @RepSinema @RepRaulGrijalva The #BlueWave will wash away the $hit of #Trumpism on Election Day @RepRubenGallego
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


The will wash away the $hit of on Election Day
{'neg': 0.304, 'neu': 0.316, 'pos': 0.381, 'compound': 0.4912}


wash away hit election day
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
GOP:MARTHA MCSALLY






@RepMcSally proud to support in any way I can. Do great things in the Senate for state of Arizona and the nation! https://t.co/JEKTeYZ1mH
{'neg': 0.0, 'neu': 0.651, 'pos': 0.349, 'compound': 0.8805}


proud to support in any way I can. Do great things in the Senate for state of Arizona and the nation! 
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


proud support way great things for arizona nation !
{'neg': 0.0, 'neu': 0.329





@RepMcSally @HouseHomeland How about reuniting migrant children who were separated from their parents because of thoughtless, inhumane, hastily enacted “zero tolerance” policy?
{'neg': 0.125, 'neu': 0.875, 'pos': 0.0, 'compound': -0.4588}


How about reuniting migrant children who were separated from their parents because of thoughtless, inhumane, hastily enacted “zero tolerance” policy?
{'neg': 0.31, 'neu': 0.443, 'pos': 0.247, 'compound': -0.4019}


reuniting migrant children were separated their parents because thoughtless inhumane hastily enacted “ zero tolerance policy
{'neg': 0.165, 'neu': 0.714, 'pos': 0.121, 'compound': -0.2023}
GOP:MARTHA MCSALLY






@RepMcSally @realDonaldTrump @WhiteHouse You are playing politics, on security and our troops, by the Republican’s failure to draft meaningful, effective legislation
{'neg': 0.114, 'neu': 0.554, 'pos': 0.332, 'compound': 0.6486}


You are playing politics, on security and our troops, by the Republican’s failure to draft mean






@ScottforFlorida @SenBillNelson Red tide Rick is running the negative campaign here
{'neg': 0.27, 'neu': 0.73, 'pos': 0.0, 'compound': -0.5719}


Red tide Rick is running the negative campaign here
{'neg': 0.0, 'neu': 0.69, 'pos': 0.31, 'compound': 0.4019}


tide running negative campaign
{'neg': 0.552, 'neu': 0.448, 'pos': 0.0, 'compound': -0.5719}
DEM:BILL NELSON






@David_Quinones @ScottforFlorida @SenBillNelson Public employees juiced thinking Gillum is going to give them tons of money, consider the source and motivation
{'neg': 0.0, 'neu': 0.893, 'pos': 0.107, 'compound': 0.34}


Public employees juiced thinking Gillum is going to give them tons of money, consider the source and motivation
{'neg': 0.552, 'neu': 0.448, 'pos': 0.0, 'compound': -0.5719}


public employees juiced thinking gillum going give them tons consider source motivation
{'neg': 0.0, 'neu': 0.821, 'pos': 0.179, 'compound': 0.34}
DEM:BILL NELSON






@FloridaRoundup @SenBillNelson  Has my vote not only fo







@dougducey AZ has 1 of the highest poverty rates in the US. And we have a very very low ranked K-12 education system. What are you doing to fix this ? @RepAndyBiggsAZ @JeffFlake @DebbieLesko @votewarren @JDMesnard @RepGosar @RepMcSally @RepRaulGrijalva @RepSinema @dg4az @RepRubenGallego
{'neg': 0.139, 'neu': 0.861, 'pos': 0.0, 'compound': -0.7159}


AZ has 1 of the highest poverty rates in the US. And we have a very very low ranked K-12 education system. What are you doing to fix this ?
{'neg': 0.619, 'neu': 0.306, 'pos': 0.075, 'compound': -0.9618}


highest poverty rates very very ranked k-12 education doing fix
{'neg': 0.268, 'neu': 0.732, 'pos': 0.0, 'compound': -0.5106}
GOP:MARTHA MCSALLY






@GOPChairwoman @realDonaldTrump The economy has been growing for 9 years. This has nothing to do with Trump. @RepAndyBiggsAZ @JeffFlake @dougducey @DebbieLesko @votewarren @JDMesnard @RepGosar @RepMcSally @westerncaucus @freedomcaucus @RepRubenGallego @RepRaulGrijalva @RepSinema @Spea

GOP:RICK SCOTT






#NovemberIsComing The Senate will be narrow but WINNABLE; here’s a list of D’s to follow and support!
TX: @BetoORourke
AZ: @kyrstensinema
NV: @RosenforNevada
MT:  @jontester
TN: @PhilBredesen
FL:  @SenBillNelson
ND: @HeidiHeitkamp
IN:   @JoeforIndiana 
MO: @clairemc
{'neg': 0.0, 'neu': 0.705, 'pos': 0.295, 'compound': 0.9324}


The Senate will be narrow but WINNABLE; here’s a list of D’s to follow and support! TX: AZ: NV: MT: TN: FL: ND: IN: MO:
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


narrow winnable ; list d follow support ! tx nv mt tn nd mo
{'neg': 0.0, 'neu': 0.491, 'pos': 0.509, 'compound': 0.807}
DEM:BILL NELSON






@Alyssa_Milano Yes..but McConnell also laid in wait by holding up Obama’s appointments not just Merrick Garland.

So many bad things happening while we watch all the public atrocities.
The Justice Department and the plan when he fires Rosenstien and Sessions.

Vote @SenBillNelson @BetoORourke
{'neg': 0.078, 'neu': 0.851, 'pos': 0

## Testing

In [ ]:
columns = ['Candidate' , 'Time' , 'Text' , 'userLocation' , 'coordinates' , 'retweeted' , 'language' , 'Sentiment' ]
AZ_path_name = 'iCloud Drive/Desktop/Politter/AZOutput'
FL_path_name = 'iCloud Drive/Desktop/Politter/FLOutput'
consume_count = 1
FL = pd.DataFrame(columns = columns)
AZ = pd.DataFrame(columns = columns)

while True:
    
    if time.localtime().tm_hour == 24:
        FL.to_csv(FL_path_name , sep = ',')
        AZ.to_csv(AZ_path_name , sep = ',')
      
    
    #Florida Race
    Rick = new.search(q="@FLGovScott" , count = consume_count  , geocode = '27.967042,-81.471955,320km' , tweet_mode = 'extended')    
    if len(Rick) == 0:
        time.sleep(60 * 15)
        Rick = new.search(q="@FLGovScott" , count = consume_count  , geocode = '27.967042,-81.471955,320km' , tweet_mode = 'extended')    
    Senter(Rick , 'GOP:RICK SCOTT' , FL)
    
    Bill = new.search(q="@SenBillNelson" , count = consume_count  , geocode = '27.967042,-81.471955,320km' , tweet_mode = 'extended')    
    if len(Bill) == 0:
        time.sleep(60 * 15)
        Bill = new.search(q="@SenBillNelson" , count = consume_count  , geocode = '27.967042,-81.471955,320km' , tweet_mode = 'extended')    
    Senter(Bill , 'DEM:BILL NELSON' , FL)
    
    
    #Arazona Race
    Martha = new.search(q="@RepMcSally" , count = consume_count  , geocode = '34.383026,-111.694738,400km' , tweet_mode = 'extended')    
    if len(Martha) == 0:
        time.sleep(60 * 15)
        Martha = new.search(q="@RepMcSally" , count = consume_count  , geocode = '34.383026,-111.694738,400km' , tweet_mode = 'extended')    
    Senter(Martha , 'GOP:MARTHA MCSALLY' , AZ)
    
    Kyrsten =  new.search(q="@RepSinema" , count = consume_count  , geocode = '34.383026,-111.694738,400km' , tweet_mode = 'extended')    
    if len(Kyrsten) == 0:
        time.sleep(60 * 15)
    Kyrsten = new.search(q="@RepSinema" , count = consume_count  , geocode = '34.383026,-111.694738,400km' , tweet_mode = 'extended')    
    Senter(Kyrsten , 'DEM:KYRSTEN SINEMA' , AZ)

In [120]:
columns = ['Candidate' , 'Time' , 'Text' , 'userLocation' , 'coordinates' , 'retweeted' , 'language' , 'Sentiment' ]
AZ_path_name = 'iCloud Drive/Desktop/Politter/AZOutput'
FL_path_name = 'iCloud Drive/Desktop/Politter/FLOutput'
consume_count = 1
FL = pd.DataFrame(columns = columns)
AZ = pd.DataFrame(columns = columns)

while True:
    
    if time.localtime().tm_hour == 24:
        FL.to_csv(FL_path_name , sep = ',')
        AZ.to_csv(AZ_path_name , sep = ',')
    Searcher("@FLGovScott" , 'GOP:RICK SCOTT' , '27.967042,-81.471955,320km' , FL)

%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                                                Text userLocation coordinates  \
0  RT @FLGovScott: Hurricane season doesn’t end u...                     None   

  retweeted language                                          Sentiment  \
0     False      NaN  {'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'comp...   

  Language  
0       en  
%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                                                Text userLocation coordinates  \
0  RT @FLGovScott: Hurricane season doesn’t end u...                     None   

  retweeted language                                          Sentiment  \
0     False      NaN  {'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'comp...   

  Language  
0       en  
%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                 

%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                                                Text userLocation coordinates  \
0  RT @FLGovScott: Hurricane season doesn’t end u...                     None   

  retweeted language                                          Sentiment  \
0     False      NaN  {'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'comp...   

  Language  
0       en  
%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                                                Text userLocation coordinates  \
0  RT @FLGovScott: Hurricane season doesn’t end u...                     None   

  retweeted language                                          Sentiment  \
0     False      NaN  {'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'comp...   

  Language  
0       en  
%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                 

%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                                                Text userLocation coordinates  \
0  RT @FLGovScott: Hurricane season doesn’t end u...                     None   

  retweeted language                                          Sentiment  \
0     False      NaN  {'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'comp...   

  Language  
0       en  
%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                                                Text userLocation coordinates  \
0  RT @FLGovScott: Hurricane season doesn’t end u...                     None   

  retweeted language                                          Sentiment  \
0     False      NaN  {'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'comp...   

  Language  
0       en  
%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                 

%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                                                Text userLocation coordinates  \
0  RT @FLGovScott: Hurricane season doesn’t end u...                     None   

  retweeted language                                          Sentiment  \
0     False      NaN  {'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'comp...   

  Language  
0       en  
%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                                                Text userLocation coordinates  \
0  RT @FLGovScott: Hurricane season doesn’t end u...                     None   

  retweeted language                                          Sentiment  \
0     False      NaN  {'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'comp...   

  Language  
0       en  
%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                 

%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                                                Text userLocation coordinates  \
0  RT @FLGovScott: Hurricane season doesn’t end u...                     None   

  retweeted language                                          Sentiment  \
0     False      NaN  {'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'comp...   

  Language  
0       en  
%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                                                Text userLocation coordinates  \
0  RT @FLGovScott: Hurricane season doesn’t end u...                     None   

  retweeted language                                          Sentiment  \
0     False      NaN  {'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'comp...   

  Language  
0       en  
%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                 

%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                                                Text userLocation coordinates  \
0  RT @FLGovScott: Hurricane season doesn’t end u...                     None   

  retweeted language                                          Sentiment  \
0     False      NaN  {'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'comp...   

  Language  
0       en  
%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                                                Text userLocation coordinates  \
0  RT @FLGovScott: Hurricane season doesn’t end u...                     None   

  retweeted language                                          Sentiment  \
0     False      NaN  {'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'comp...   

  Language  
0       en  
%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                 

%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                                                Text userLocation coordinates  \
0  RT @FLGovScott: Hurricane season doesn’t end u...                     None   

  retweeted language                                          Sentiment  \
0     False      NaN  {'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'comp...   

  Language  
0       en  
%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                                                Text userLocation coordinates  \
0  RT @FLGovScott: Hurricane season doesn’t end u...                     None   

  retweeted language                                          Sentiment  \
0     False      NaN  {'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'comp...   

  Language  
0       en  
%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                 

%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                                                Text userLocation coordinates  \
0  RT @FLGovScott: Hurricane season doesn’t end u...                     None   

  retweeted language                                          Sentiment  \
0     False      NaN  {'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'comp...   

  Language  
0       en  
%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                                                Text userLocation coordinates  \
0  RT @FLGovScott: Hurricane season doesn’t end u...                     None   

  retweeted language                                          Sentiment  \
0     False      NaN  {'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'comp...   

  Language  
0       en  
%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                 

KeyboardInterrupt: 

In [ ]:
   Rick = new.search(q="@FLGovScott" , count = consume_count  , geocode = '27.967042,-81.471955,320km' , tweet_mode = 'extended')    
    if len(Rick) == 0:
        time.sleep(60 * 15)
        Rick = new.search(q="@FLGovScott" , count = consume_count  , geocode = '27.967042,-81.471955,320km' , tweet_mode = 'extended')    
    Senter(Rick , 'GOP:RICK SCOTT' , FL)
    